In [18]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator



In [19]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                   zoom_range=0.2,
                                   horizontal_flip=True)

In [20]:
test_datagen = ImageDataGenerator(rescale=1./255)


In [21]:
xtrain = train_datagen.flow_from_directory('Training',
                                           target_size=(64,64),
                                           class_mode='categorical',
                                           batch_size=200)


Found 6385 images belonging to 2 classes.


In [22]:
xtest = test_datagen.flow_from_directory('Testing',
                                         target_size=(64,64),
                                         class_mode='categorical',
                                         batch_size=200)


Found 1829 images belonging to 2 classes.


In [23]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout


In [24]:
model = Sequential() # Initializing sequential model
model.add(Conv2D(32,(3,3),activation='relu',input_shape=(64,64,3))) # convolution layer
model.add(Conv2D(32,(3,3),activation='relu')) # convolution layer
model.add(MaxPooling2D(pool_size=(2, 2))) # Max pooling layer
model.add(Dropout(0.25)) # Dropout layer


In [25]:
model.add(Conv2D(64,(3,3),activation='relu')) # convolution layer
model.add(Conv2D(64,(3,3),activation='relu')) # convolution layer
model.add(MaxPooling2D(pool_size=(2, 2))) # Max pooling layer
model.add(Dropout(0.25)) # Dropout layer


In [26]:
model.add(Conv2D(128,(3,3),activation='relu')) # convolution layer
model.add(Conv2D(128,(3,3),activation='relu')) # convolution layer
model.add(MaxPooling2D(pool_size=(2, 2))) # Max pooling layer
model.add(Dropout(0.25)) # Dropout layer


In [27]:
model.add(Flatten()) # Flatten layer
model.add(Dense(512,activation='relu')) # Hidden layer 1
model.add(Dropout(0.5)) # Dropout layer
model.add(Dense(2,activation='softmax'))


In [28]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
model.summary()


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 62, 62, 32)        896       
                                                                 
 conv2d_7 (Conv2D)           (None, 60, 60, 32)        9248      
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 30, 30, 32)       0         
 2D)                                                             
                                                                 
 dropout_4 (Dropout)         (None, 30, 30, 32)        0         
                                                                 
 conv2d_8 (Conv2D)           (None, 28, 28, 64)        18496     
                                                                 
 conv2d_9 (Conv2D)           (None, 26, 26, 64)        36928     
                                                      

In [29]:
xtrain.class_indices

{'Fake1': 0, 'Original': 1}

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

early_stop = EarlyStopping(monitor='val_accuracy', patience=5)

lr = ReduceLROnPlateau(monitor='val_accuracy', factor=0.5, min_lr=0.00001)

callback = [early_stop,lr]

r=model.fit_generator(xtrain,steps_per_epoch=len(xtrain),
                    epochs=100,
                    callbacks=callback,
                    validation_data=xtest,
                    validation_steps=len(xtest))


C:\Users\admin\AppData\Local\Temp\ipykernel_16636\905664577.py:9: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  r=model.fit_generator(xtrain,steps_per_epoch=len(xtrain),


Epoch 1/100
32/32 [==============================] - 113s 3s/step - loss: 0.5630 - accuracy: 0.7729 - val_loss: 0.5102 - val_accuracy: 0.7977 - lr: 0.0010
Epoch 2/100
32/32 [==============================] - 108s 3s/step - loss: 0.5193 - accuracy: 0.7865 - val_loss: 0.5036 - val_accuracy: 0.7977 - lr: 0.0010
Epoch 3/100
32/32 [==============================] - 109s 3s/step - loss: 0.5129 - accuracy: 0.7865 - val_loss: 0.4985 - val_accuracy: 0.7977 - lr: 0.0010
Epoch 4/100
32/32 [==============================] - 110s 3s/step - loss: 0.5155 - accuracy: 0.7865 - val_loss: 0.4966 - val_accuracy: 0.7977 - lr: 0.0010
Epoch 5/100
32/32 [==============================] - 109s 3s/step - loss: 0.5057 - accuracy: 0.7865 - val_loss: 0.5020 - val_accuracy: 0.7977 - lr: 0.0010
Epoch 6/100
27/32 [========================>.....] - ETA: 16s - loss: 0.4973 - accuracy: 0.7913

In [14]:
model.save('boat.h5')

In [15]:
from tensorflow.keras.preprocessing import image
import numpy as np
from sklearn.metrics import classification_report

# predict classes for test set
y_pred = model.predict(xtest)
y_pred = np.argmax(y_pred, axis=1)

# get true classes for test set
y_true = xtest.classes

# print classification report
print(classification_report(y_true, y_pred, target_names=xtest.class_indices.keys()))


10/10 [==============================] - 6s 537ms/step
              precision    recall  f1-score   support

       Fake1       0.80      1.00      0.89      1459
    Original       0.00      0.00      0.00       370

    accuracy                           0.80      1829
   macro avg       0.40      0.50      0.44      1829
weighted avg       0.64      0.80      0.71      1829



C:\Users\admin\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\admin\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\admin\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [16]:
import matplotlib.pyplot as plt
img = image.load_img('test.jpg',target_size=(64,64)) # Reading image
plt.show('/Downloads/test.jpg')
x = image.img_to_array(img) # Converting image into array
x = np.expand_dims(x,axis=0)# expanding Dimensions
pred = np.argmax(model.predict(x)) # Predicting the higher probablity index
op = ['original','fake'] # Creating list
op[pred] # List indexing with output

1/1 [==============================] - 0s 206ms/step


'original'

In [17]:
img = image.load_img('Training/Fake1/2(168).png',target_size=(64,64)) # Reading image
x = image.img_to_array(img) # Converting image into array
x = np.expand_dims(x,axis=0) # expanding Dimensions
pred = np.argmax(model.predict(x)) # Predicting the higher probablity index
op = ['original','fake'] # Creating list
op[pred] # List indexing with output

1/1 [==============================] - 0s 38ms/step


'original'